# 🎯 Sistema Sussy - Entrenamiento Unificado (Colab / Lambda / Local)

Este notebook permite entrenar modelos de detección jerárquica en **múltiples plataformas**:

| Plataforma | GPU | Uso recomendado |
|------------|-----|-----------------|
| **Google Colab** | T4/L4 gratis | Pruebas rápidas, entrenamiento pequeño |
| **Lambda Labs** | A10/A100 | Entrenamiento serio, datasets grandes |
| **Local CPU** | - | Verificar pipeline, pruebas mínimas |

## 📋 Pasos:
1. Conectar GPU (Colab: Runtime > T4/L4 | Lambda: ya incluida)
2. Preparar dataset (extraer frames + anotar)
3. Configurar el entrenamiento (auto-detecta hardware)
4. Entrenar y exportar modelo ONNX

**IMPORTANTE:** Ejecuta las celdas en orden. El sistema detecta automáticamente el entorno.

In [ ]:
# 1️⃣ Detectar Entorno y Configurar
import os
import platform

# Detectar plataforma
def detectar_plataforma():
    if os.environ.get("COLAB_GPU") or "google.colab" in str(globals()):
        return "COLAB"
    elif os.environ.get("LAMBDA_TASK_ROOT") or "lambda" in platform.node().lower():
        return "LAMBDA"
    elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
        return "KAGGLE"
    else:
        return "LOCAL"

PLATAFORMA = detectar_plataforma()
print(f"🖥️ Plataforma detectada: {PLATAFORMA}")

# Verificar GPU
try:
    result = os.popen('nvidia-smi --query-gpu=name,memory.total --format=csv,noheader').read()
    if result.strip():
        print(f"\n🎮 GPU(s) detectadas:")
        for line in result.strip().split('\n'):
            print(f"   {line}")
except:
    print("⚠️ No se detectó nvidia-smi")

import torch
print(f"\n📦 PyTorch version: {torch.__version__}")
print(f"🔧 CUDA available: {torch.cuda.is_available()}")

# Test real de CUDA
CUDA_FUNCIONAL = False
if torch.cuda.is_available():
    try:
        test = torch.zeros(1, device="cuda")
        del test
        CUDA_FUNCIONAL = True
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"✅ CUDA version: {torch.version.cuda}")
        print(f"✅ CUDA funcional: ¡Sí!")
    except Exception as e:
        print(f"⚠️ GPU detectada pero CUDA no funcional: {e}")
        print("   Usaremos CPU para entrenamiento")

DEVICE = "cuda" if CUDA_FUNCIONAL else "cpu"
print(f"\n🎯 Device a usar: {DEVICE}")

In [ ]:
# Instalar Ultralytics (YOLO)
!pip install -q ultralytics

from ultralytics import YOLO
print(f"\n✅ Ultralytics instalado correctamente")

In [ ]:
# 2️⃣ Configurar Almacenamiento (adapta a plataforma)
import os

if PLATAFORMA == "COLAB":
    # Google Drive para persistencia
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        WORK_DIR = "/content/drive/MyDrive/Sussy_Training"
    except:
        print("⚠️ No se pudo montar Drive, usando almacenamiento local")
        WORK_DIR = "/content/sussy_training"
        
elif PLATAFORMA == "LAMBDA":
    # Lambda: usar almacenamiento local (persiste entre sesiones si usas /home)
    WORK_DIR = os.path.expanduser("~/sussy_training")
    
elif PLATAFORMA == "KAGGLE":
    WORK_DIR = "/kaggle/working/sussy_training"
    
else:  # LOCAL
    # Detectar si estamos en el proyecto Sussy
    if os.path.exists("sussy"):
        WORK_DIR = "./training_output"
    else:
        WORK_DIR = os.path.expanduser("~/sussy_training")

# Crear estructura de carpetas
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(f"{WORK_DIR}/datasets", exist_ok=True)
os.makedirs(f"{WORK_DIR}/models", exist_ok=True)
os.makedirs(f"{WORK_DIR}/frames", exist_ok=True)  # Para frames extraídos

print(f"\n📁 Carpeta de trabajo: {WORK_DIR}")
print(f"   📂 datasets/  - Tus datasets anotados")
print(f"   📂 models/    - Modelos entrenados")
print(f"   📂 frames/    - Frames extraídos de vídeos")

# Mostrar espacio disponible
try:
    import shutil
    total, used, free = shutil.disk_usage(WORK_DIR)
    print(f"\n💾 Espacio disponible: {free // (2**30)} GB")
except:
    pass  # No crítico si falla

## 🎬 Extracción de Frames desde Vídeos

**¿Hay que recortar los objetos?**

| Fase | Formato de imagen | ¿Recortar? |
|------|-------------------|------------|
| **Fase 1: Detección** | Imagen COMPLETA + bounding box | ❌ No |
| **Fases 2-3: Clasificación** | CROP del objeto | ✅ Sí |
| **Fase 4: Atributos** | CROP del objeto | ✅ Sí |

**¿Frames similares o variados?**
- ✅ Saltar 5-10 frames entre capturas (evita redundancia)
- ✅ Incluir diferentes ángulos, iluminación, fondos
- ✅ Incluir oclusiones parciales
- ❌ Evitar 100 frames idénticos del mismo objeto
- 💡 **Ratio ideal:** ~30% similares + ~70% variados

**¿Cómo entrenar atributos (ej: insignia militar)?**
```
Imagen → Detector Persona → Crop Persona → Clasificador Atributos
                                              ├── rol: militar
                                              ├── rango: cabo (detectado por insignia)
                                              └── equipamiento: casco, chaleco...
```
Entrena con la **persona completa** que lleva la insignia. El modelo aprende a buscar la zona relevante.

In [ ]:
# 🎬 EXTRAER FRAMES DE VÍDEO (opcional)
# ======================================
# Sube un vídeo y extrae frames inteligentemente

import cv2
import numpy as np
from pathlib import Path

def extraer_frames_video(
    path_video: str,
    path_salida: str,
    skip_frames: int = 10,      # Extraer 1 de cada N frames
    min_blur: float = 100.0,    # Filtrar frames borrosos
    max_frames: int = None,     # Límite máximo (None = todos)
):
    """Extrae frames de un vídeo con filtrado de calidad."""
    
    path_salida = Path(path_salida)
    path_salida.mkdir(parents=True, exist_ok=True)
    
    cap = cv2.VideoCapture(path_video)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"📹 Vídeo: {Path(path_video).name}")
    print(f"   {total} frames, {fps:.1f} FPS, {total/fps:.1f} segundos")
    
    frame_idx = 0
    saved = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_idx += 1
        
        # Saltar frames
        if frame_idx % skip_frames != 0:
            continue
        
        # Filtrar borrosos (varianza del Laplaciano)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blur_score = cv2.Laplacian(gray, cv2.CV_64F).var()
        
        if blur_score < min_blur:
            continue
        
        # Guardar
        timestamp = frame_idx / fps
        nombre = f"frame_{frame_idx:06d}_t{timestamp:.2f}s.jpg"
        cv2.imwrite(str(path_salida / nombre), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
        saved += 1
        
        if max_frames and saved >= max_frames:
            break
    
    cap.release()
    print(f"✅ Extraídos {saved} frames en: {path_salida}")
    return saved

# =====================================
# EJEMPLO DE USO:
# =====================================
# 1. Sube tu vídeo a Colab/Lambda
# 2. Descomenta y ejecuta:

# extraer_frames_video(
#     "/content/mi_video.mp4",           # Ruta al vídeo
#     f"{WORK_DIR}/frames/mi_video/",    # Carpeta de salida
#     skip_frames=10,                    # 1 de cada 10 frames
#     min_blur=100.0,                    # Filtrar borrosos
#     max_frames=500,                    # Máximo 500 frames
# )

print("💡 Descomenta el código de arriba para extraer frames de tu vídeo")
print("   O sube vídeos a la carpeta y ejecuta en batch:")

## 3️⃣ Preparar Dataset

**Estructura para Detección (imagen completa + anotaciones):**
```
datasets/mi_dataset/
├── data.yaml           # Configuración (clases, rutas)
├── train/
│   ├── images/         # Imágenes completas
│   └── labels/         # Archivos .txt con bounding boxes
└── val/
    ├── images/
    └── labels/
```

**Estructura para Clasificación (crops organizados por clase):**
```
datasets/mi_clasificador/
├── train/
│   ├── clase1/         # Carpeta por clase
│   └── clase2/
└── val/
    ├── clase1/
    └── clase2/
```

In [ ]:
# Descargar dataset de ejemplo para probar (COCO128)
print("📥 Descargando dataset de ejemplo...")
!wget -q https://ultralytics.com/assets/coco128.zip -O /content/coco128.zip
!unzip -q /content/coco128.zip -d /content/

print("✅ Dataset COCO128 listo en /content/coco128/")
print("   (128 imágenes con 80 clases - perfecto para probar)")

In [ ]:
# 4️⃣ CONFIGURACIÓN DEL ENTRENAMIENTO
# =====================================
# El sistema ajusta automáticamente según tu hardware

# ============ CONFIGURA ESTO ============
TIPO = "detect"  # "detect" o "classify"
DATASET = "coco128.yaml"  # Tu dataset: f"{WORK_DIR}/datasets/tu_dataset/data.yaml"
PROYECTO = "sussy_prueba_v1"

# Preset: "prueba" (rápido), "desarrollo" (balance), "produccion" (serio)
PRESET = "prueba" if PLATAFORMA == "LOCAL" or not CUDA_FUNCIONAL else "desarrollo"
# =========================================

# Configuración automática según preset y hardware
PRESETS = {
    "prueba": {
        "epochs": 10, "batch_size": 4, "img_size": 320,
        "patience": 5, "modelo": "yolo11n.pt"
    },
    "desarrollo": {
        "epochs": 50, "batch_size": 16, "img_size": 640,
        "patience": 15, "modelo": "yolo11s.pt"
    },
    "produccion": {
        "epochs": 100, "batch_size": 32, "img_size": 640,
        "patience": 20, "modelo": "yolo11m.pt"
    },
    "lambda_full": {
        "epochs": 200, "batch_size": 32, "img_size": 960,
        "patience": 30, "modelo": "yolo11l.pt"
    },
}

# Usar Lambda_full si estamos en Lambda
if PLATAFORMA == "LAMBDA":
    PRESET = "lambda_full"

config = PRESETS[PRESET]
EPOCHS = config["epochs"]
BATCH_SIZE = config["batch_size"]
IMG_SIZE = config["img_size"]
PATIENCE = config["patience"]
MODELO_BASE = config["modelo"]
LEARNING_RATE = 0.01
AUGMENT = True

# Ajuste de batch si hay poca memoria GPU (< 8GB)
if CUDA_FUNCIONAL:
    try:
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
        if gpu_mem < 8:
            BATCH_SIZE = min(BATCH_SIZE, 8)
            print(f"⚠️ GPU con {gpu_mem:.1f}GB: batch reducido a {BATCH_SIZE}")
    except:
        pass

print(f"\n📋 Configuración ({PRESET.upper()}):")
print(f"   🎯 Tipo: {TIPO}")
print(f"   📂 Dataset: {DATASET}")
print(f"   🤖 Modelo: {MODELO_BASE}")
print(f"   🔢 Epochs: {EPOCHS}")
print(f"   📦 Batch: {BATCH_SIZE}")
print(f"   🖼️ ImgSize: {IMG_SIZE}")
print(f"   ⏱️ Patience: {PATIENCE}")
print(f"   🔧 Device: {DEVICE}")
print(f"\n💡 Modifica PRESET arriba para cambiar la intensidad del entrenamiento")

In [ ]:
# 5️⃣ ENTRENAR MODELO
# =====================
from ultralytics import YOLO
import time

print(f"🔄 Cargando modelo base: {MODELO_BASE}")
model = YOLO(MODELO_BASE)

# Determinar device
train_device = 0 if DEVICE == "cuda" else "cpu"
print(f"\n🚀 Iniciando entrenamiento en: {DEVICE.upper()}")
print(f"   Epochs: {EPOCHS}, Batch: {BATCH_SIZE}")
print("=" * 50)

start_time = time.time()

try:
    results = model.train(
        data=DATASET,
        epochs=EPOCHS,
        imgsz=IMG_SIZE,
        batch=BATCH_SIZE,
        lr0=LEARNING_RATE,
        patience=PATIENCE,
        augment=AUGMENT,
        project=f"{WORK_DIR}/models",
        name=PROYECTO,
        exist_ok=True,
        device=train_device,
        verbose=True,
        half=DEVICE == "cuda",  # FP16 solo en GPU
        workers=4 if DEVICE == "cuda" else 2,
    )
    
    elapsed = time.time() - start_time
    print("\n" + "=" * 50)
    print(f"✅ ¡Entrenamiento completado en {elapsed/60:.1f} minutos!")
    print(f"   Modelo guardado en: {WORK_DIR}/models/{PROYECTO}/")
    
except Exception as e:
    print(f"\n❌ Error durante entrenamiento: {e}")
    print("\n💡 Posibles soluciones:")
    print("   - Reducir BATCH_SIZE")
    print("   - Usar PRESET = 'prueba'")
    print("   - Verificar que el dataset existe")

In [ ]:
# 6️⃣ VER RESULTADOS DE ENTRENAMIENTO
from IPython.display import Image, display
import os

results_dir = f"{WORK_DIR}/models/{PROYECTO}"

# Mostrar curvas de entrenamiento
results_png = os.path.join(results_dir, "results.png")
if os.path.exists(results_png):
    print("📊 Curvas de entrenamiento:")
    display(Image(filename=results_png, width=900))

# Mostrar matriz de confusión (si existe)
confusion_png = os.path.join(results_dir, "confusion_matrix.png")
if os.path.exists(confusion_png):
    print("\n📊 Matriz de confusión:")
    display(Image(filename=confusion_png, width=700))

In [ ]:
# 7️⃣ EXPORTAR A ONNX (para tu RTX 5080)
# =========================================
best_model_path = f"{WORK_DIR}/models/{PROYECTO}/weights/best.pt"

if os.path.exists(best_model_path):
    print("📦 Exportando modelo a ONNX...")
    
    model = YOLO(best_model_path)
    
    # Exportar a ONNX
    onnx_path = model.export(format="onnx", imgsz=IMG_SIZE, simplify=True)
    
    # Tamaños de archivo
    pt_size = os.path.getsize(best_model_path) / 1024 / 1024
    onnx_size = os.path.getsize(onnx_path) / 1024 / 1024
    
    print(f"\n✅ Modelos exportados:")
    print(f"   PyTorch: {best_model_path} ({pt_size:.1f} MB)")
    print(f"   ONNX:    {onnx_path} ({onnx_size:.1f} MB)")
    print(f"\n💡 El archivo ONNX funciona con ONNX Runtime en tu RTX 5080")
else:
    print("⚠️ No se encontró el modelo entrenado. Ejecuta el entrenamiento primero.")

In [ ]:
# 8️⃣ PROBAR INFERENCIA CON EL MODELO
# =====================================
import matplotlib.pyplot as plt
import cv2

best_model_path = f"{WORK_DIR}/models/{PROYECTO}/weights/best.pt"

if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    # Buscar imagen de prueba
    test_img = "/content/coco128/images/train2017/000000000009.jpg"
    
    if os.path.exists(test_img):
        print(f"🖼️ Probando inferencia...")
        
        # Inferencia
        results = model.predict(test_img, imgsz=IMG_SIZE, conf=0.25)
        
        # Mostrar resultado
        result_img = results[0].plot()
        result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(12, 8))
        plt.imshow(result_img)
        plt.axis('off')
        plt.title(f'Detecciones: {len(results[0].boxes)}')
        plt.show()
        
        # Listar detecciones
        print(f"\n📋 Objetos detectados: {len(results[0].boxes)}")
        for box in results[0].boxes:
            cls = int(box.cls.item())
            conf = box.conf.item()
            name = model.names[cls]
            print(f"   - {name}: {conf:.2%}")
else:
    print("⚠️ Entrena el modelo primero")

In [ ]:
# 9️⃣ DESCARGAR MODELO (alternativa a Google Drive)
# ===================================================
from google.colab import files

best_model_path = f"{WORK_DIR}/models/{PROYECTO}/weights/best.pt"
onnx_path = best_model_path.replace(".pt", ".onnx")

print("📥 Descargando modelos al PC...")

if os.path.exists(best_model_path):
    files.download(best_model_path)
    print(f"   ✅ {best_model_path}")
    
if os.path.exists(onnx_path):
    files.download(onnx_path)
    print(f"   ✅ {onnx_path}")

print("\n💡 También puedes copiar desde Google Drive: Sussy_Training/models/")

---

## 📝 Notas Importantes

### Para tu propio dataset:

1. **Detección (Fase 1):** Sube imágenes completas + archivos `.txt` con bounding boxes
2. **Clasificación (Fases 2-3):** Sube crops organizados en carpetas por clase

### Formato de anotación YOLO (archivo .txt):
```
# clase x_centro y_centro ancho alto (todo normalizado 0-1)
0 0.5 0.5 0.2 0.3
1 0.25 0.75 0.1 0.15
```

### Recomendaciones:
- **Pruebas rápidas:** 30-50 epochs con yolo11n.pt
- **Entrenamiento serio:** 100-300 epochs con yolo11m.pt o yolo11l.pt
- **Si hay error de memoria:** Reduce BATCH_SIZE (16 → 8 → 4)

### Próximos pasos:
1. ✅ Probar este notebook con COCO128
2. ⬜ Extraer frames de tus vídeos con `extraer_frames.py`
3. ⬜ Anotar imágenes (usa [Label Studio](https://labelstud.io/) o [CVAT](https://cvat.ai/))
4. ⬜ Subir dataset a Google Drive
5. ⬜ Entrenar con tu dataset
6. ⬜ Exportar ONNX e integrar en Sistema Sussy

---
*Sistema Sussy - Entrenamiento Jerárquico v1.0*